In [ ]:
%run app.py

/rds/general/user/abs121/home/anaconda3/envs/vid-sam-4/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://b7f09107406ab8053b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


get meta information of input video
No module named 'spatial_correlation_sampler'
Failed to import PyTorch Correlation, For better efficiency, please install it.
No module named 'spatial_correlation_sampler'
Failed to import PyTorch Correlation, For better efficiency, please install it.
No module named 'spatial_correlation_sampler'
Failed to import PyTorch Correlation, For better efficiency, please install it.
SegTracker has been initialized
Everything
Start tracking!
processed frame 26, obj_num 6
finished
frame 26 writed
/rds/general/user/abs121/home/UROP_2024/Segment-and-Track-Anything-1.6/tracking_results/A1_8-5s/A1_8-5s_seg.mp4 saved

finished
/rds/general/user/abs121/home/UROP_2024/Segment-and-Track-Anything-1.6/tracking_results/A1_8-5s/A1_8-5s_seg.gif saved
updating: rds/general/user/abs121/home/UROP_2024/Segment-and-Track-Anything-1.6/tracking_results/A1_8-5s/A1_8-5s_masks/ (stored 0%)
updating: rds/general/user/abs121/home/UROP_2024/Segment-and-Track-Anything-1.6/tracking_resul

Traceback (most recent call last):
  File "/rds/general/user/abs121/home/anaconda3/envs/vid-sam-4/lib/python3.11/site-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/rds/general/user/abs121/home/anaconda3/envs/vid-sam-4/lib/python3.11/site-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/rds/general/user/abs121/home/anaconda3/envs/vid-sam-4/lib/python3.11/site-packages/gradio/blocks.py", line 1897, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/rds/general/user/abs121/home/anaconda3/envs/vid-sam-4/lib/python3.11/site-packages/gradio/blocks.py", line 1483, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/rds/general/user/

Start tracking!
processed frame 26, obj_num 6
finished
frame 26 writed
/rds/general/user/abs121/home/UROP_2024/Segment-and-Track-Anything-1.6/tracking_results/A1_8-5s/A1_8-5s_seg.mp4 saved

finished
/rds/general/user/abs121/home/UROP_2024/Segment-and-Track-Anything-1.6/tracking_results/A1_8-5s/A1_8-5s_seg.gif saved
updating: rds/general/user/abs121/home/UROP_2024/Segment-and-Track-Anything-1.6/tracking_results/A1_8-5s/A1_8-5s_masks/ (stored 0%)
updating: rds/general/user/abs121/home/UROP_2024/Segment-and-Track-Anything-1.6/tracking_results/A1_8-5s/A1_8-5s_masks/00013.png (deflated 42%)
updating: rds/general/user/abs121/home/UROP_2024/Segment-and-Track-Anything-1.6/tracking_results/A1_8-5s/A1_8-5s_masks/00017.png (deflated 42%)
updating: rds/general/user/abs121/home/UROP_2024/Segment-and-Track-Anything-1.6/tracking_results/A1_8-5s/A1_8-5s_masks/00018.png (deflated 41%)
updating: rds/general/user/abs121/home/UROP_2024/Segment-and-Track-Anything-1.6/tracking_results/A1_8-5s/A1_8-5s_masks/